In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc
import time

In [3]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import server

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [4]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


In [5]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [24]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer04 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2020-08-06

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Socia

In [25]:
def mkdir_transfer_check(dir_from, dir_to, sftp, ssh_t, str_expect='[rh183', verbose=1, timeout_cp=10, timeout_diff=10):
    """
    Makes a directory
    Transfers
    Checks to make sure the transfer worked
    """
    ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives
    
    sftp.mkdir_p(dir_to)
    
    if verbose>1:
        print(f'copy starting,         time: {time.ctime()}, dir:{dir_from}')
    ssh_t.send(f"cp -r {dir_from}/. {dir_to}/.")
    test = ssh_t.expect(str_expect, verbose=0, total_timeout=timeout_cp)  ## wait for copy to end
    if test[0] is None:
        print('copy expect timed out')
    elif 'No such file or directory' in test[0]:
        print(f'copy failed: {test[0]}')
    elif test[1]==False:
        if verbose>0:
            print(f'Post copy expect timed out. time: {time.ctime()}, dir: {dir_from}')
    else:
        if verbose>1:
            print(f'copy expect completed. time: {time.ctime()}, dir: {dir_from}')

    diff = ssh_t.send(f"diff -qr {dir_from} {dir_to}") ## check differences in directory
    test = ssh_t.expect('Only in', verbose=0, total_timeout=timeout_diff)
    if test[0] is None:
        print('diff expect timed out')
    elif 'No such file or directory' in test[0]:
        print(f'diff check failed: {test[0]}')
    elif verbose>0:
        if test[1]:
            print(f"differences found: {test}")
        else:
            print(f"diff check passes: No differences found. time: {time.ctime()}, from dir: {dir_from} to dir: {dir_to}")
    
    ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives

In [26]:
# ## make directory structure on MICROSCOPE
# animals = ['AEG21', 'AEG22']
# dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
# methods_MICROSCOPE = ['clean_traces', 'ROI_classification_logisticRegression', 'spike_inference', 'suite2p_output']
# methods_o2 = ['cleanTraces', 'classify_rois', 'spike_inference', 'suite2p_output']

In [27]:
# ## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
# dir_data_o2 = '/n/data1/hms/neurobio/sabatini/rich/analysis/'
# dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

# [[[mkdir_transfer_check(
#     dir_from=str(Path(dir_data_o2) / method_o2 / animal / date / 'jobNum_0'),
#     dir_to=str(Path(dir_data_MICROSCOPE) / animal / date / method_MICROSCOPE),
#     sftp=sftp,
#     ssh_t=ssh_t,
#     str_expect='[rh183',
# ) for method_o2,method_MICROSCOPE in zip(methods_o2, methods_MICROSCOPE)] for date in dates] for animal in animals];

In [28]:
## make directory structure on MICROSCOPE

animals = ['AEG21', 'AEG22']
# animals = ['AEG21']
dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
# dates = ['2022_05_13']

In [ ]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_from = '/n/files/Neurobio/MICROSCOPE/Priya/FaceRhythms'
dir_data_to = '/n/files/Neurobio/MICROSCOPE/Rich/data/2pRAM/face_rhythm_motor_mapping'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

[[mkdir_transfer_check(
    dir_from=str(Path(dir_data_from) / animal / (date+'_'+animal)),
    dir_to=str(Path(dir_data_to) / animal / date / 'camera'),
    sftp=sftp,
    ssh_t=ssh_t,
    str_expect='[rh183',
    verbose=1,
    timeout_cp=60*45,
    timeout_diff=60*15
) for date in dates] for animal in animals];

In [23]:
# ssh_t.close()